# Question generation serving on Sagemaker
Code is from https://colab.research.google.com/drive/1_2_mS5l29QHI1pXaqa4YLzAO5xm-HmH9?usp=sharing

In [5]:
!pip install -q sagemaker sagemaker[local]
!pip install gdown

# Install `transformers` from master
!pip install -q git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers|sagemaker'

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Stored in directory: /Users/hunkim/Library/Caches/pip/wheels/32/8e/8d/0fec7d933759a0880dc34be487e1d92c51ba8fdf4e9f265ff7
Successfully built gdown
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
sagemaker           1.60.2    
tokenizers          0.7.0     
transformers        2.11.0    
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-que-gen'
role = "arn:aws:iam::294038372338:role/hunkimSagemaker"

In [8]:
!gdown -O  t5_que_gen.zip --id 1vhsDOW9wUUO83IQasTPlkxb82yxmMH-V
!unzip t5_que_gen.zip


Downloading...
From: https://drive.google.com/uc?id=1vhsDOW9wUUO83IQasTPlkxb82yxmMH-V
To: /Users/hunkim/work/sagemaker-aihub/que_gen/t5_que_gen.zip
1.65GB [01:09, 23.8MB/s]
Archive:  t5_que_gen.zip
replace t5_que_gen_model/t5_base_tok_que_gen/spiece.model? [y]es, [n]o, [A]ll, [N]one, [r]ename:^C
tar: t5_que_gen_modelt5_que_gen_model: Cannot stat: No such file or directory
tar: Error exit delayed from previous errors.


In [12]:
!mkdir -p model
!tar cvfz model/model.tar.gz t5_ans_gen_model t5_que_gen_model

a t5_ans_gen_model
a t5_ans_gen_model/t5_base_ans_gen
a t5_ans_gen_model/t5_base_tok_ans_gen
a t5_ans_gen_model/t5_base_tok_ans_gen/added_tokens.json
a t5_ans_gen_model/t5_base_tok_ans_gen/tokenizer_config.json
a t5_ans_gen_model/t5_base_tok_ans_gen/special_tokens_map.json
a t5_ans_gen_model/t5_base_tok_ans_gen/spiece.model
a t5_ans_gen_model/t5_base_ans_gen/config.json
a t5_ans_gen_model/t5_base_ans_gen/pytorch_model.bin
a t5_que_gen_model
a t5_que_gen_model/t5_base_tok_que_gen
a t5_que_gen_model/logs.zip
a t5_que_gen_model/t5_base_que_gen
a t5_que_gen_model/t5_base_que_gen/config.json
a t5_que_gen_model/t5_base_que_gen/pytorch_model.bin
a t5_que_gen_model/t5_base_tok_que_gen/added_tokens.json
a t5_que_gen_model/t5_base_tok_que_gen/tokenizer_config.json
a t5_que_gen_model/t5_base_tok_que_gen/special_tokens_map.json
a t5_que_gen_model/t5_base_tok_que_gen/spiece.model


In [13]:
model = sagemaker_session.upload_data(path='model', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(model+"/model.tar.gz"))
model_data = model+"/model.tar.gz"

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-que-gen/model.tar.gz


In [14]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [22]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data=model_data,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='que_gen.py',
                     source_dir='code',
                     predictor_cls=JSONPredictor)

In [34]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')
#predictor = model.deploy(initial_instance_count=1, instance_type='local')

# Get the end point
endpoint = predictor.endpoint
print(endpoint)

------------------!pytorch-inference-2020-06-12-03-31-58-602


In [35]:
text = """
Sunghun Kim is an Associate Professor of Computer Science at the Hong Kong University of Science and Technology. 

His core research area is Software Engineering and Machine Learning 
focusing on software evolution, program analysis and empirical studies. 
He publishes his work on top venues such as TSE, ICSE, FSE, AAAI, SOSP and ISSTA. 
He is a four-time winner of the ACM SIGSOFT Distinguished Paper Award 
(ICSE 2007, ASE 2012, ICSE 2013 and ISSTA 2014). 
Besides, he received various awards including 2010 and 
2011 Microsoft Software Innovation Awards and 2011 Google Faculty Research Award. 
Further information is available at http://www.cse.ust.hk/~hunkim.
"""
input = {
    'text': text
}
response = predictor.predict(input)
print(response)

[{'answer': 'Hong Kong University of Science and Technology', 'question': 'Where is Sunghun Kim an Associate Professor of Computer Science?'}, {'answer': 'Software Engineering and Machine Learning', 'question': "What is Sunghun Kim's core research area?"}, {'answer': 'TSE, ICSE, FSE, AAAI, SOSP and ISSTA', 'question': 'Where does Sunghun Kim publish his work?'}, {'answer': 'ACM SIGSOFT Distinguished Paper Award', 'question': 'What award has Sunghun Kim won four times?'}, {'answer': 'Microsoft Software Innovation Awards', 'question': 'What awards did Kim receive in 2010 and 2011?'}, {'answer': 'http://www.cse.ust.hk/ ⁇ hunkim', 'question': 'Where can you find more information about Sunghun Kim?'}]


In [37]:
import boto3
import json

channy="""
Channy Yun is a Principal Technical Evangelist at Amazon Web Services and works with Korean developers to enable them to use AWS Cloud services. He focuses on engaging technical audiences such as developers, communities, and AWS Korea User Group with over 17,000 members and shared knowledge of the latest AWS cloud services in blog posts, social media, and public speaking. He has a special interest in cloud native architecture such as Serverless and Microservices based on DevOps.
"""
client = boto3.client('sagemaker-runtime')

input = {
    'text': channy
}
payload = json.dumps(input)

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType="application/json",
    Accept="application/json" ,
    Body=payload
)

print(response['Body'].read())  

b'[{"answer": "Channy Yun", "question": "Who is the Principal Technical Evangelist at Amazon Web Services?"}, {"answer": "17,000", "question": "How many members does the AWS Korea User Group have?"}, {"answer": "Cloud native architecture", "question": "What type of architecture does Channy Yun have a special interest in?"}]'


In [33]:
sagemaker_session.delete_endpoint(predictor.endpoint)